# Reading Google Sheets File That Contain Match Infos

In [1]:
import pandas as pd

# UEFA Google Sheets URL
csv_url = "https://docs.google.com/spreadsheets/d/1WfEG-1icUjj6k7TGePJQEXH-w0TLEIcN/export?format=csv"

# Read the Google Sheet as a DataFrame
df = pd.read_csv(csv_url, dtype={'FTHG': 'Int64', 'FTAG': 'Int64', 'HTHG': 'Int64', 'HTAG': 'Int64'})
df.head()

,Date,League,Home,Away,FTHG,FTAG,HTHG,HTAG
0,9/17/2024,UCL,Juventus,PSV,3,1,2,0
1,9/17/2024,UCL,Young Boys,Aston Villa,0,3,0,2
2,9/17/2024,UCL,Bayern,Dinamo Zagreb,9,2,3,0
3,9/17/2024,UCL,Milan,Liverpool,1,3,1,2
4,9/17/2024,UCL,Real Madrid,Stuttgart,3,1,0,0


# Separating Previous and Future Matches and Tournaments

In [2]:
# Separate rows with NA values
next_matches = df[df.isna().any(axis=1)]
next_leagues = next_matches['League'].unique().tolist()

# Separate rows without NA values
previous_matches = df[~df.isna().any(axis=1)]

next_matches.head()

,Date,League,Home,Away,FTHG,FTAG,HTHG,HTAG
301,14/11/2024,UNL,Belgium,Italy,<NA>,<NA>,<NA>,<NA>
302,14/11/2024,UNL,France,Israel,<NA>,<NA>,<NA>,<NA>
303,14/11/2024,UNL,Greece,England,<NA>,<NA>,<NA>,<NA>
304,14/11/2024,UNL,Ireland,Finland,<NA>,<NA>,<NA>,<NA>
305,14/11/2024,UNL,Kazakhstan,Austria,<NA>,<NA>,<NA>,<NA>


# Calculating Functions Needed For Dixon-Coles Model

In [3]:
from scipy.optimize import minimize
from scipy.stats import poisson
import numpy as np

def rho_correction(x, y, lambda_x, mu_y, rho):
    if x==0 and y==0:
        return 1- (lambda_x * mu_y * rho)
    elif x==0 and y==1:
        return 1 + (lambda_x * rho)
    elif x==1 and y==0:
        return 1 + (mu_y * rho)
    elif x==1 and y==1:
        return 1 - rho
    else:
        return 1.0

def dc_log_like(x, y, alpha_x, beta_x, alpha_y, beta_y, rho, gamma):
    lambda_x, mu_y = np.exp(alpha_x + beta_y + gamma), np.exp(alpha_y + beta_x) 
    return (np.log(rho_correction(x, y, lambda_x, mu_y, rho)) + 
            np.log(poisson.pmf(x, lambda_x)) + np.log(poisson.pmf(y, mu_y)))



def solve_parameters(dataset, half_or_full = 'full', debug = False, init_vals=None, options={'disp': True, 'maxiter':100},
                     constraints = [{'type':'eq', 'fun': lambda x: sum(x[:20])-20}] , **kwargs):
    teams = np.sort(dataset['Home'].unique())
    # check for no weirdness in dataset
    away_teams = np.sort(dataset['Away'].unique())
    if not np.array_equal(teams, away_teams):
        raise ValueError("Something's not right")
    n_teams = len(teams)
    if init_vals is None:
        # random initialisation of model parameters
        init_vals = np.concatenate((np.random.uniform(0,1,(n_teams)), # attack strength
                                      np.random.uniform(0,-1,(n_teams)), # defence strength
                                      np.array([0, 1.0]) # rho (score correction), gamma (home advantage)
                                     ))

    def estimate_paramters(params):
        score_coefs = dict(zip(teams, params[:n_teams]))
        defend_coefs = dict(zip(teams, params[n_teams:(2*n_teams)]))
        rho, gamma = params[-2:]
        if half_or_full == 'full':
            log_like = [dc_log_like(row.FTHG, row.FTAG, score_coefs[row.Home], defend_coefs[row.Home],
                        score_coefs[row.Away], defend_coefs[row.Away], rho, gamma) for row in dataset.itertuples()]
        elif half_or_full == 'half':
            log_like = [dc_log_like(row.HTHG, row.HTAG, score_coefs[row.Home], defend_coefs[row.Home],
                        score_coefs[row.Away], defend_coefs[row.Away], rho, gamma) for row in dataset.itertuples()]

        return -sum(log_like)
    opt_output = minimize(estimate_paramters, init_vals, options=options, constraints = constraints, **kwargs)
    if debug:
        # sort of hacky way to investigate the output of the optimisation process
        return opt_output
    else:
        return dict(zip(["attack_"+team for team in teams] + 
                        ["defence_"+team for team in teams] +
                        ['rho', 'home_adv'],
                        opt_output.x))

# Calculating Lambda Values for Dixon-Coles Model

In [4]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
stats_df = pd.DataFrame()
full_time_models = []
half_time_models = []

for league in next_leagues:
    league_df = previous_matches[previous_matches['League'] == league]
    
    full_time_estimates = solve_parameters(league_df, half_or_full = 'full')
    full_time_models.append(full_time_estimates)

    half_time_estimates = solve_parameters(league_df, half_or_full = 'half')
    half_time_models.append(half_time_estimates)

C:\Users\99451\AppData\Local\Temp\ipykernel_13648\342759101.py:19: RuntimeWarning: invalid value encountered in log
  return (np.log(rho_correction(x, y, lambda_x, mu_y, rho)) +
C:\Users\99451\AppData\Local\Temp\ipykernel_13648\342759101.py:20: RuntimeWarning: divide by zero encountered in log
  np.log(poisson.pmf(x, lambda_x)) + np.log(poisson.pmf(y, mu_y)))


Optimization terminated successfully    (Exit mode 0)
            Current function value: 240.28724660948092
            Iterations: 63
            Function evaluations: 7060
            Gradient evaluations: 63
Optimization terminated successfully    (Exit mode 0)
            Current function value: 155.7027616584062
            Iterations: 77
            Function evaluations: 8590
            Gradient evaluations: 77


# Calculating Probability Matrices for HT/FT

In [5]:
#First Function needs work to make it more understandable and a df rather than matrix!
def dixon_coles_simulate_match(params_dict, homeTeam, awayTeam, max_goals=10):
    team_avgs = [np.exp(params_dict['attack_'+homeTeam] + params_dict['defence_'+awayTeam] + params_dict['home_adv']),
                 np.exp(params_dict['defence_'+homeTeam] + params_dict['attack_'+awayTeam])]
    team_pred = [[poisson.pmf(i, team_avg) for i in range(0, max_goals+1)] for team_avg in team_avgs]
    output_matrix = np.outer(np.array(team_pred[0]), np.array(team_pred[1]))
    correction_matrix = np.array([[rho_correction(home_goals, away_goals, team_avgs[0],
                                                   team_avgs[1], params_dict['rho']) for away_goals in range(2)]
                                   for home_goals in range(2)])
    output_matrix[:2,:2] = output_matrix[:2,:2] * correction_matrix
    return output_matrix

full_time_matrices = []
half_time_matrices = []

for i in range(len(next_matches)):
    my_league = next_matches['League'].iloc[i]
    league_index = next_leagues.index(my_league)
    ft_match_score_matrix = dixon_coles_simulate_match(full_time_models[league_index], 
                                                       next_matches['Home'].iloc[i], next_matches['Away'].iloc[i], max_goals = 8)
    ht_match_score_matrix = dixon_coles_simulate_match(half_time_models[league_index], 
                                                       next_matches['Home'].iloc[i], next_matches['Away'].iloc[i], max_goals = 4)
    full_time_matrices.append(ft_match_score_matrix)
    half_time_matrices.append(ht_match_score_matrix)

# Calculating Probabilities of Dixon-Coles Model

In [6]:
ft1, ftx, ft2, ft_score = [], [], [], []
over_15, over_25, under_35, under_45, btts = [], [], [], [], []
ht1, htx, ht2, ht_score, ht_over05, ht_under15 = [], [], [], [], [], []
ho05, ao05, ho15, ao15, hu25, au25 = [], [], [], [], [], []

# Helper function to calculate total goals for each score
def total_goals(i, j):
    return i + j

for i in range(len(next_matches)):
    my_matrix = full_time_matrices[i]
    ht_matrix = half_time_matrices[i]

    ft1.append(round(np.sum(np.tril(my_matrix, k=-1)) * 100, 2)) # Sum of lower triangular values (home win)
    ftx.append(round(np.sum(np.diag(my_matrix)) * 100, 2)) # Sum of diagonal values (draw)
    ft2.append(round(np.sum(np.triu(my_matrix, k=1)) * 100, 2)) # Sum of higher triangular values (away_win)
    
    max_score = np.unravel_index(np.argmax(my_matrix), my_matrix.shape) # Find the index of the maximum score
    home_goals, away_goals = max_score
    ft_score.append(f"{home_goals}-{away_goals}") # Format the score as 'home-away'

    # Calculate the probabilities
    over_15.append(round(np.sum([my_matrix[i, j] for i in range(my_matrix.shape[0]) for j in range(my_matrix.shape[1]) if total_goals(i, j) > 1.5]) * 100, 2))
    over_25.append(round(np.sum([my_matrix[i, j] for i in range(my_matrix.shape[0]) for j in range(my_matrix.shape[1]) if total_goals(i, j) > 2.5]) * 100, 2))
    under_35.append(round(np.sum([my_matrix[i, j] for i in range(my_matrix.shape[0]) for j in range(my_matrix.shape[1]) if total_goals(i, j) <= 3.5]) * 100, 2))
    under_45.append(round(np.sum([my_matrix[i, j] for i in range(my_matrix.shape[0]) for j in range(my_matrix.shape[1]) if total_goals(i, j) <= 4.5]) * 100, 2))

    # Calculate BTTS (both teams to score and goals != 0)
    btts.append(round(np.sum([my_matrix[i, j] for i in range(1, my_matrix.shape[0]) for j in range(1, my_matrix.shape[1])]) * 100, 2)) 

    # Calculate statistics for Half Time
    ht1.append(round(np.sum(np.tril(ht_matrix, k=-1)) * 100, 2)) # Sum of lower triangular values (home win)
    htx.append(round(np.sum(np.diag(ht_matrix)) * 100, 2)) # Sum of diagonal values (draw)
    ht2.append(round(np.sum(np.triu(ht_matrix, k=1)) * 100, 2)) # Sum of higher triangular values (away_win)

    ht_max_score = np.unravel_index(np.argmax(ht_matrix), ht_matrix.shape) # Find the index of the maximum score
    ht_hogs, ht_awgs = ht_max_score
    ht_score.append(f"{ht_hogs}-{ht_awgs}") # Format the score as 'home-away'

    ht_over05.append(round(np.sum([ht_matrix[i, j] for i in range(ht_matrix.shape[0]) for j in range(ht_matrix.shape[1]) if total_goals(i, j) > 0.5]) * 100, 2))   
    ht_under15.append(round(np.sum([ht_matrix[i, j] for i in range(ht_matrix.shape[0]) for j in range(ht_matrix.shape[1]) if total_goals(i, j) < 1.5]) * 100, 2)) 

    ho05.append(round(np.sum(my_matrix[1:,:]) * 100, 2))
    ao05.append(round(np.sum(my_matrix[:,1:]) * 100, 2))
    ho15.append(round(np.sum(my_matrix[2:,:]) * 100, 2))
    ao15.append(round(np.sum(my_matrix[:,2:]) * 100, 2))
    hu25.append(round(np.sum(my_matrix[:3,:]) * 100, 2))
    au25.append(round(np.sum(my_matrix[:,:3]) * 100, 2))
    

# Combine lists into a DataFrame
final_results = pd.DataFrame({
    'League': next_matches['League'], 'Home': next_matches['Home'], 'Away': next_matches['Away'],
    'FT1': ft1, 'FTX': ftx, 'FT2': ft2, 'FTR': ft_score,
    'DC1X': [x + y for x, y in zip(ft1, ftx)], 'DC12': [x + y for x, y in zip(ft1, ft2)], 'DCX2': [x + y for x, y in zip(ftx, ft2)],
    '1.5O': over_15, '2.5O': over_25, '3.5U': under_35, '4.5U': under_45, 'BTTS': btts,
    'HT1': ht1, 'HTX': htx, 'HT2': ht2, 'HTR': ht_score,
    'HTDC1X': [x + y for x, y in zip(ht1, htx)], 'HTDC12': [x + y for x, y in zip(ht1, ht2)], 'HTDCX2': [x + y for x, y in zip(htx, ht2)],
    'HT0.5O': ht_over05, 'HT1.5U': ht_under15, 'H0.5O':ho05, 'A0.5O':ao05, 'H1.5O':ho15, 'A1.5O':ao15, 'H2.5U':hu25, 'A2.5U':au25
})

# Function to highlight values higher than threshold
def highlight_values(value):
    if isinstance(value, str):
        return ''  # Return empty string for NaN values
    elif value > 70:
    #color = 'red'
        return 'background-color: red'
    else:
        return ''

# Apply the style
with pd.option_context('display.precision', 2):
    styled_df = final_results.style.applymap(highlight_values)
styled_df.to_excel("UEFA.xlsx", index = False)
# Display the styled DataFrame
from IPython.display import display, HTML
display(styled_df)

C:\Users\99451\AppData\Local\Temp\ipykernel_13648\1302854748.py:74: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_df = final_results.style.applymap(highlight_values)


,League,Home,Away,FT1,FTX,FT2,FTR,DC1X,DC12,DCX2,1.5O,2.5O,3.5U,4.5U,BTTS,HT1,HTX,HT2,HTR,HTDC1X,HTDC12,HTDCX2,HT0.5O,HT1.5U,H0.5O,A0.5O,H1.5O,A1.5O,H2.5U,A2.5U
301,UNL,Belgium,Italy,35.190000,21.720000,43.090000,0-1,56.910000,78.280000,64.810000,76.450000,54.630000,67.700000,83.710000,56.250000,4.430000,44.500000,50.030000,0-0,48.930000,54.460000,94.530000,73.290000,41.330000,73.950000,78.160000,38.910000,44.930000,84.650000,80.340000
302,UNL,France,Israel,89.110000,4.450000,2.680000,4-0,93.560000,91.790000,7.130000,92.710000,85.730000,23.140000,39.750000,52.470000,63.090000,31.780000,1.070000,2-0,94.870000,64.160000,32.850000,79.410000,28.250000,95.060000,53.370000,89.830000,18.710000,18.010000,91.550000
303,UNL,Greece,England,75.190000,17.540000,7.250000,1-0,92.730000,82.440000,24.790000,63.810000,37.990000,81.730000,92.640000,24.250000,0.000000,100.000000,0.000000,0-0,100.000000,0.000000,100.000000,0.000000,100.000000,84.340000,29.890000,55.320000,4.990000,71.580000,99.410000
304,UNL,Ireland,Finland,57.510000,25.180000,17.320000,1-0,82.690000,74.830000,42.500000,56.060000,30.530000,86.890000,95.310000,31.030000,0.000000,45.360000,54.510000,0-0,45.360000,54.510000,99.870000,54.510000,81.220000,74.250000,43.830000,39.310000,11.430000,84.380000,97.910000
305,UNL,Kazakhstan,Austria,0.000000,8.140000,91.750000,0-2,8.140000,91.750000,99.890000,71.330000,45.720000,75.570000,89.000000,0.000000,0.000000,34.850000,64.700000,0-1,34.850000,64.700000,99.550000,64.700000,71.580000,0.000000,91.750000,0.000000,71.330000,99.880000,54.160000
306,UNL,Slovenia,Norway,41.680000,22.980000,35.340000,1-0,64.660000,77.020000,58.320000,71.560000,48.120000,73.630000,87.760000,51.100000,13.710000,60.640000,25.620000,0-0,74.350000,39.330000,86.260000,54.820000,68.770000,74.510000,70.820000,39.670000,34.880000,84.140000,87.250000
307,UNL,Armenia,Faroe Islands,73.410000,13.940000,12.510000,2-0,87.350000,85.920000,26.450000,85.670000,68.390000,52.930000,71.900000,55.570000,15.840000,56.060000,27.900000,0-0,71.900000,43.740000,83.960000,66.690000,48.600000,92.330000,61.140000,72.860000,24.450000,52.180000,92.780000
308,UNL,North Macedonia,Latvia,92.870000,5.910000,0.870000,2-0,98.780000,93.740000,6.780000,80.590000,58.950000,63.090000,80.280000,9.980000,27.940000,67.610000,4.440000,0-0,95.550000,32.380000,72.050000,36.380000,89.840000,94.460000,10.750000,79.110000,0.600000,43.250000,99.630000
309,UNL,Portugal,Poland,81.570000,10.090000,7.810000,3-0,91.660000,89.380000,17.900000,90.020000,76.390000,42.370000,61.900000,55.540000,43.030000,48.170000,7.920000,0-0,91.200000,50.950000,56.090000,73.500000,37.640000,95.220000,58.700000,81.800000,22.330000,38.860000,93.350000
310,UNL,Scotland,Croatia,35.010000,25.350000,39.640000,0-1,60.360000,74.650000,64.990000,63.290000,38.320000,81.500000,92.510000,43.140000,27.650000,54.290000,17.690000,0-0,81.940000,45.340000,71.980000,70.850000,39.950000,65.520000,68.560000,28.810000,32.180000,90.740000,88.860000
